In [92]:
import numpy as np

In [93]:
# |0> = [1,0], |1> = [0,1]
H = np.array([[1,1],[1,-1]])/np.sqrt(2)

In [94]:
def invQFT(f):
    """
    Performs inverse quantum fourier transform on a tensor products of qubit states.

    Intputs:
    f - tensor product of qubit states, in form [[qubit 1 state], [qubit 2 state], ...]
    """

    N = f.shape[0]
    fInv = np.empty([N,2],dtype=np.complex_)

    for i in range(N): 
        s = f[i]
        for k in range(2,N-i):
            s[1] *= np.exp(-2*np.pi*complex(0,1)/(2**k))
        s = H @ s
        fInv[i] = s
    
    return fInv


In [95]:
def findNumber(s): #rename to convert from binary
    """
    Function that finds the associated number for a qubit tensor product state. 
    """
    binaryForm = np.empty(s.shape[0])
    for i,ket in enumerate(s):
        if np.allclose(ket,np.array([1,0])):
            binaryForm[i] = 0
        elif np.allclose(ket,np.array([0,1])):
            binaryForm[i] = 1
        else:
            print("error")
    
    number = sum([b*2**i for i,b in enumerate(np.flip(binaryForm))])
    return number

        

In [96]:
#test case for invQFT: 3 qubit system, |2> = |010> state should be the result
invf = np.array([[1,np.exp(np.pi*complex(0,0.5))],[1,-1],[1,1]])/np.sqrt(2) 
print(findNumber(invQFT(invf)))



2.0


In [103]:

def phaseEstimation(t,U,e): 
    """
    Performs phase estimation.

    Inputs:
    t - accuracy of psi returned
    U - unitary matrix
    e - normalised ket
    """
    N = U.shape[0]

    #cache?
    def pow(n,prev=np.identity(N)):
        next = U @ prev
        if n == 1:
            return next
        return pow(n-1,next)
    

    f = np.zeros([t,2])

    for i in range(t):
        s = np.array([1,0])
        s = H @ s

        #apply 
        s[1] *= e @ pow(2**i) @ e
        
        f[t-1-i] = s

    return findNumber(invQFT(f)) #apply inverse QFT


In [104]:
#test for phase esimation, identity matrix, result should be 1
U1 = np.array([[1,0],[0,1]])
e1 = np.array([1,0])
phaseEstimation(2,U1,e1)

0.0

In [102]:



def findOrder(a,N,n,m,t): 
    """
    a - some int (will be removed as a parameter eventually)
    N - number of qubits used 
    n - number to be factorized 
    m - number of common factors found
    """
    #use basis |0> = [1,0,0,..], |1> = [0,1,0,..], ...
    #define U_a
    U_a = np.zeros([2**N,2**N])
    for i in range(2**N):
        for j in range(2**N):
            if i == a*j % n:
                U_a[i,j] = 1
    
    #in the actual algorithm |1> is plugged in, however due to this implementation not being able to consider
    #superpositions the following is considered
    eigenVal, eigenVec = np.linalg.eig(U_a)
    rs = np.empty(m) 
    for i in range(m):
        index = np.random.randint(0,high=2**N)
        rs[i] = index/phaseEstimation(t,U_a,eigenVec[index])
        
    #find LCM
    R = np.lcm.reduce(rs)
    
    return R

    



IndentationError: expected an indented block (1175582580.py, line 3)